# Phase 1: Corporate & Organizational Intelligence Report

Target: Bitmine Immersion Technologies (BMNR) Status: High Risk / High Variance Core Finding: BMNR is attempting a massive pivot from a small Bitcoin miner to a global "Ethereum Treasury," modeled after MicroStrategy but for ETH.

## Tool 1: The "Real-Time" Dilution Monitor
This script parses SEC filings to find the most recent "Shares Outstanding" count.

In [5]:
import requests
import pandas as pd
import json

def get_real_time_share_count():
    # BMNR CIK Code
    cik_str = "0001829311" 
    
    # 1. Use the SEC "Company Facts" API (The Source of Truth)
    # This JSON contains every single number BMNR has ever reported to the government.
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik_str}.json"
    
    headers = {'User-Agent': 'PierreAlessandria/1.0 (paales01@gmail.com)'}
    
    try:
        r = requests.get(url, headers=headers)
        if r.status_code != 200:
            return "❌ Connection Error to SEC Database."
        
        data = r.json()
        
        # 2. Extract "EntityCommonStockSharesOutstanding"
        # This is the specific accounting tag for share count.
        try:
            shares_data = data['facts']['dei']['EntityCommonStockSharesOutstanding']['units']['shares']
        except KeyError:
            return "❌ Could not find share count data in XBRL."

        # 3. Sort by date to get the absolute latest report
        df = pd.DataFrame(shares_data)
        df = df.sort_values(by='end', ascending=False)
        
        latest_report = df.iloc[0]
        
        count = int(latest_report['val'])
        date = latest_report['end']
        form = latest_report['form']
        
        return f"""
        🔍 SEC DATA VERIFIED
        ---------------------------
        Latest Share Count: {count:,}
        Report Date:        {date}
        Source Document:    {form}
        ---------------------------
        """
        
    except Exception as e:
        return f"Script Error: {e}"

print(get_real_time_share_count())


        🔍 SEC DATA VERIFIED
        ---------------------------
        Latest Share Count: 384,067,823
        Report Date:        2025-11-20
        Source Document:    10-K
        ---------------------------
        


The "Deep-Dive" Python Script to find the "Hidden" Shares

This script specifically targets the DEF 14A (where companies must list the voting share count for upcoming meetings) and 8-K (where they announce new issuance).



In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# Latest share count

def fetch_latest_share_count_from_text():
    # 1. SETUP: Define Headers (Required by SEC) and Target
    cik = "0001829311" # BMNR
    email = "your_email@example.com" # Replace with your email for SEC compliance
    headers = {'User-Agent': f'ResearchBot/1.0 ({email})'}
    
    # 2. GET FILING LIST
    # We pull the specific "Submission" history JSON from the SEC
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    
    print(f"📡 Connecting to SEC EDGAR for CIK: {cik}...")
    try:
        r = requests.get(url, headers=headers)
        if r.status_code != 200: return "Error connecting to SEC."
        
        data = r.json()
        filings = pd.DataFrame(data['filings']['recent'])
        
        # 3. FILTER FOR "DEF 14A" (Proxy) and "8-K" (Current Reports)
        # These are the documents where share counts hide in plain sight
        target_forms = ['DEF 14A', '8-K', 'S-1'] 
        relevant_filings = filings[filings['form'].isin(target_forms)].head(10)
        
        print(f"📚 Scanning last 10 relevant filings for 'Outstanding Shares'...")
        print("-" * 60)

        found_count = False

        for index, row in relevant_filings.iterrows():
            # Build the document URL
            acc_num = row['accessionNumber'].replace('-', '')
            doc_name = row['primaryDocument']
            doc_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{acc_num}/{doc_name}"
            
            # Fetch the document text
            doc_resp = requests.get(doc_url, headers=headers)
            soup = BeautifulSoup(doc_resp.content, 'html.parser')
            
            # Normalize text (remove newlines/weird spacing for better regex matching)
            page_text = " ".join(soup.get_text().split())
            
            # 4. THE REGEX "HUNTERS"
            # Pattern A (Common in Proxy/DEF 14A): "On the Record Date... 425,841,924 shares"
            pattern_proxy = r"(\d{1,3}(?:,\d{3})*)\s+shares\s+of\s+(?:our\s+)?common\s+stock\s+outstanding"
            
            # Pattern B (Common in 8-K/S-1): "As of [Date], X shares were issued and outstanding"
            pattern_standard = r"(\d{1,3}(?:,\d{3})*)\s+shares\s+were\s+issued\s+and\s+outstanding"
            
            match_proxy = re.search(pattern_proxy, page_text, re.IGNORECASE)
            match_standard = re.search(pattern_standard, page_text, re.IGNORECASE)
            
            detected_count = None
            
            if match_proxy:
                detected_count = match_proxy.group(1)
            elif match_standard:
                detected_count = match_standard.group(1)
                
            if detected_count:
                print(f"✅ FOUND in {row['form']} (Filed: {row['filingDate']})")
                print(f"   Link: {doc_url}")
                print(f"   Excerpt: \"...{detected_count} shares of common stock outstanding...\"")
                print(f"   👉 COUNT: {detected_count}")
                print("-" * 60)
                found_count = True
                # We stop after finding the first (latest) valid number to avoid confusion
                break 
        
        if not found_count:
            print("⚠️ No exact text match found in recent documents. They may use non-standard phrasing.")

    except Exception as e:
        print(f"Script Error: {e}")

# Run the Scraper
fetch_latest_share_count_from_text()

📡 Connecting to SEC EDGAR for CIK: 0001829311...
📚 Scanning last 10 relevant filings for 'Outstanding Shares'...
------------------------------------------------------------
✅ FOUND in DEF 14A (Filed: 2025-12-09)
   Link: https://www.sec.gov/Archives/edgar/data/0001829311/000149315225026868/formdef14a.htm
   Excerpt: "...425,841,924 shares of common stock outstanding..."
   👉 COUNT: 425,841,924
------------------------------------------------------------
📡 Scanning BMNR filings for 'Treasury Updates'...


New Tool: The "Treasury Tracker" Script

Since BMNR updates its holdings in unstructured text (Press Releases) rather than neat data tables (XBRL), you need a script that "reads" the news for you.

This Python script monitors 8-K filings for the specific keywords: "ETH", "Cash", and "Holdings".

In [15]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

def track_bmnr_treasury_deep():
    # 1. SETUP
    cik = "0001829311"
    headers = {'User-Agent': 'PierreAlessandria/1.0 (paales01@gmail.com)'} 
    
    # Get the filing history
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    print(f"📡 Deep-Scanning BMNR Filings (focusing on Exhibit 99.1)...")
    
    try:
        r = requests.get(url, headers=headers)
        if r.status_code != 200: return
        data = r.json()
        filings = pd.DataFrame(data['filings']['recent'])
        
        # Look at last 5 8-Ks
        recent_8ks = filings[filings['form'] == '8-K'].head(5)
        
        for index, row in recent_8ks.iterrows():
            acc_num = row['accessionNumber'].replace('-', '')
            
            # TRICK: Check the 'Index' page to find the Exhibit 99.1 file
            index_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{acc_num}/index.json"
            
            index_resp = requests.get(index_url, headers=headers)
            if index_resp.status_code != 200: continue
            
            files = index_resp.json()['directory']['item']
            
            # Target the Press Release (usually ex99-1.htm)
            target_files = [f['name'] for f in files if '.htm' in f['name'] and ('ex99' in f['name'].lower())]
            
            for file_name in target_files:
                doc_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{acc_num}/{file_name}"
                doc_text = requests.get(doc_url, headers=headers).text
                soup = BeautifulSoup(doc_text, 'html.parser')
                text = " ".join(soup.get_text().split())

                # --- VERIFIED PATTERNS (Based on Jan 5, 2026 Filing) ---
                # Matches: "4,144 million ETH" OR "4,143,502 ETH"
                eth_match = re.search(r"(\d{1,3}(?:,\d{3})*|\d+\.\d+\s+million)\s+(?:ETH|Ethereum|tokens)", text, re.IGNORECASE)
                
                # Matches: "total cash of $915 million" OR "$915 million in cash"
                cash_match = re.search(r"total\s+cash\s+of\s+\$(\d+(?:\.\d+)?)\s+(?:million|billion)", text, re.IGNORECASE)
                
                if eth_match or cash_match:
                    print(f"\n✅ FOUND DATA in Exhibit 99.1 (Filed {row['filingDate']})")
                    print(f"   🔗 Link: {doc_url}")
                    
                    if eth_match: 
                        print(f"   💎 ETH Found: {eth_match.group(1)} ETH")
                    if cash_match: 
                        print(f"   💵 Cash Found: ${cash_match.group(1)} Million")
                    return # Stop after finding the latest data

    except Exception as e:
        print(f"Error: {e}")

track_bmnr_treasury_deep()

📡 Deep-Scanning BMNR Filings (focusing on Exhibit 99.1)...

✅ FOUND DATA in Exhibit 99.1 (Filed 2026-01-05)
   🔗 Link: https://www.sec.gov/Archives/edgar/data/0001829311/000149315226000274/ex99-1.htm
   💎 ETH Found: 4.144 Million ETH
   💵 Cash Found: $915 Million


## Tool 2: The NAV Premium/Discount Calculator
This is your "Buy/Sell" signal. If BMNR trades at a 2x premium to its ETH holdings, it's risky.

In [16]:
import yfinance as yf

def calculate_bmnr_nav_premium():
    # INPUTS (Update these from Tool 1 results)
    shares_outstanding = 425_841_924 
    eth_holdings = 4_144_000  # From Phase 1 Intelligence
    cash_on_hand = 915_000_000 # $ Cash from recent reports
    
    # 1. Get Live Market Data
    bmnr = yf.Ticker("BMNR")
    eth = yf.Ticker("ETH-USD")
    
    try:
        current_stock_price = bmnr.history(period="1d")['Close'].iloc[-1]
        current_eth_price = eth.history(period="1d")['Close'].iloc[-1]
    except:
        return "Error fetching market data."

    # 2. Calculate Values
    market_cap = current_stock_price * shares_outstanding
    crypto_assets_value = eth_holdings * current_eth_price
    total_nav = crypto_assets_value + cash_on_hand
    nav_per_share = total_nav / shares_outstanding
    
    # 3. Calculate Premium/Discount
    premium = (market_cap - total_nav) / total_nav
    
    print(f"--- BMNR VALUATION DASHBOARD ---")
    print(f"Stock Price:       ${current_stock_price:.2f}")
    print(f"Implied NAV/Share: ${nav_per_share:.2f}")
    print(f"ETH Price Used:    ${current_eth_price:.2f}")
    print(f"----------------------------------")
    
    if premium > 0:
        return f"⚠️ TRADING AT PREMIUM: +{premium:.2%} (Overvalued relative to assets)"
    else:
        return f"✅ TRADING AT DISCOUNT: {premium:.2%} (Undervalued relative to assets)"

print(calculate_bmnr_nav_premium())

--- BMNR VALUATION DASHBOARD ---
Stock Price:       $30.06
Implied NAV/Share: $32.20
ETH Price Used:    $3088.15
----------------------------------
✅ TRADING AT DISCOUNT: -6.65% (Undervalued relative to assets)
